Use two different machine learning models-
1. SVM
2. Naive Bayes

Overview of steps coming forth
<ul>
<li>A little bit of feature engineering</li>
<li>2 different Models
<ul><li>SVM</li>
<li>Naive Bayes</li>
</ul>
</li>
<li>Evaluation Metrics chosen
<ul><li>precision: % of selected items that are correct = tp/(tp+fp) (tp: True Positive, fp: False Positive)</li>
<li>Recall: % of correct items that are selected = tp/(tp+fn) (tp: True Positive, fn: False Negative)</li>
</ul>
</li>

<li>Model comparisons</li>
</ul>

I have improved the features by playing with the words in the overviews which represent textual data. For that I have used TF-IDf to assign weightage to every word in the bag of words. sklearn provides this facility via TfidfTransformer.

In [46]:
import pickle
import numpy as np
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import make_scorer
from sklearn.metrics import classification_report
import tmdbsimple as tmdb
posterFolder='posters_final/'

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer

In [11]:
f1=open('X.pckl','rb')
f2=open('Y.pckl','rb')
X=pickle.load(f1)
Y=pickle.load(f2)
f1.close()
f2.close()
del f1
del f2

In [12]:
tfidfTransformer = TfidfTransformer()
XTfidf = tfidfTransformer.fit_transform(X)
XTfidf.shape

(1559, 1344)

Divide X and Y into train and test split

In [14]:
msk = np.random.rand(XTfidf.shape[0]) < 0.8

In [15]:
msk

array([False,  True, False, ...,  True,  True,  True], dtype=bool)

In [24]:
f=open('movies_with_overviews.pckl','rb')
moviesWithOverviews=pickle.load(f)
f.close()
del f

f=open('Genredict.pckl','rb')
GenreIDToName = pickle.load(f)
f.close()
del f

In [20]:
XTrainTfidf=XTfidf[msk]
XTestTfidf=XTfidf[~msk]
YTrain=Y[msk]
YTest=Y[~msk]
positions=range(len(moviesWithOverviews))
# print positions
testMovies=np.asarray(positions)[~msk]
#testMovies

# Training SVM Model

In [25]:
parameters = {'kernel':['linear'], 'C':[0.01, 0.1, 1.0]}
gridCV = GridSearchCV(SVC(class_weight='balanced'), parameters, scoring=make_scorer(f1_score, average='micro'))
classif = OneVsRestClassifier(gridCV)

classif.fit(XTrainTfidf, YTrain)
GenreIDToName

{12: u'Adventure',
 14: u'Fantasy',
 16: u'Animation',
 18: u'Drama',
 27: u'Horror',
 28: u'Action',
 35: u'Comedy',
 36: u'History',
 37: u'Western',
 53: u'Thriller',
 80: u'Crime',
 99: u'Documentary',
 878: u'Science Fiction',
 9648: u'Mystery',
 10402: u'Music',
 10749: u'Romance',
 10751: u'Family',
 10752: u'War',
 10769: 'Foreign',
 10770: u'TV Movie'}

In [26]:
genreNames=[u'Adventure',
u'Fantasy',
u'Animation',
u'Drama',
u'Horror',
u'Action',
u'Comedy',
u'History',
u'Western',
u'Thriller',
u'Crime',
u'Documentary',
u'Science Fiction',
u'Mystery',
u'Music',
u'Romance',
u'Family',
u'War',
'Foreign',
u'TV Movie']

In [27]:
predstfidf=classif.predict(XTestTfidf)

print classification_report(YTest, predstfidf, target_names=genreNames)

                 precision    recall  f1-score   support

      Adventure       0.40      0.52      0.45        60
        Fantasy       0.43      0.51      0.47        51
      Animation       0.42      0.54      0.47        41
          Drama       0.58      0.62      0.60       124
         Horror       0.42      0.48      0.44        42
         Action       0.25      1.00      0.40        75
         Comedy       0.00      0.00      0.00        70
        History       0.09      1.00      0.17        28
        Western       0.33      0.12      0.18         8
       Thriller       0.44      0.63      0.52        82
          Crime       0.38      0.64      0.47        33
    Documentary       0.55      0.50      0.52        12
Science Fiction       0.58      0.42      0.49        50
        Mystery       0.26      0.28      0.27        32
          Music       0.86      0.58      0.69        33
        Romance       0.00      0.00      0.00        43
         Family       0.56    

/home/paritosh/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Looking at our model's predictions for a small sample of movies from our test set, 

In [30]:
genreList=sorted(list(GenreIDToName.keys()))

In [31]:
predictions=[]
for i in range(XTestTfidf.shape[0]):
    predGenres=[]
    movieLabelScores=predstfidf[i]
#     print movieLabelScores
    for j in range(20):
        #print j
        if movieLabelScores[j]!=0:
            genre=GenreIDToName[genreList[j]]
            predGenres.append(genre)
    predictions.append(predGenres)

In [32]:
#f=open('classifer_svc','wb')
#pickle.dump(classif,f)
#f.close()
#del f

In [35]:
for i in range(XTestTfidf.shape[0]):
    if i%60==0 and i!=0:
        print 'MOVIE: ',moviesWithOverviews[i]['title'],'\tPREDICTION: ',','.join(predictions[i])

MOVIE:  Justin and the Knights of Valour 	PREDICTION:  Action,History,Thriller
MOVIE:  Igor 	PREDICTION:  Drama,Action,History,Thriller,Crime
MOVIE:  Terminus 	PREDICTION:  Adventure,Fantasy,Action,History
MOVIE:  Donkey X 	PREDICTION:  Action,History,Documentary,Music
MOVIE:  Les Misérables 	PREDICTION:  Action,History,Thriller,Crime,Family


# Training Naive Bayes Model

In [36]:
from sklearn.naive_bayes import MultinomialNB
classifnb = OneVsRestClassifier(MultinomialNB())
classifnb.fit(X[msk].toarray(), YTrain)
predsnb=classifnb.predict(X[~msk].toarray())

In [37]:
#f=open('classifer_nb','wb')
#pickle.dump(classifnb,f)
#f.close()
#del f

In [38]:
predictionsnb=[]
for i in range(XTestTfidf.shape[0]):
    predGenres=[]
    movieLabelScores=predsnb[i]
    for j in range(20):
        #print j
        if movieLabelScores[j]!=0:
            genre=GenreIDToName[genreList[j]]
            predGenres.append(genre)
    predictionsnb.append(predGenres)

In [48]:
apiKey='b2db12a77bd83b7fa87202026d7f7ca5'
tmdb.API_KEY = apiKey
searchInstance = tmdb.Search()
def get_poster(movie):
    searchResponse = searchInstance.movie(query=movie)
    id=searchResponse['results'][0]['id']
    movie = tmdb.Movies(id)
    posterPath=movie.info()['poster_path']
    title=movie.info()['original_title']
    url='image.tmdb.org/t/p/original'+posterPath
    title='_'.join(title.split(' '))
    strCmd='wget -O '+posterFolder+title+'.jpg '+url
    os.system(strCmd)

def get_movie_id(movie):
    searchResponse = searchInstance.movie(query=movie)
    movieId=searchResponse['results'][0]['id']
    return movieId

def get_movie_info(movie):
    searchResponse = searchInstance.movie(query=movie)
    id=searchResponse['results'][0]['id']
    movie=tmdb.Movies(id)
    info=movie.info()
    return info

def get_movie_genres(movie):
    searchResponse = searchInstance.movie(query=movie)
    id=searchResponse['results'][0]['id']
    movie = tmdb.Movies(id)
    genres=movie.info()['genres']
    return genres

def get_overview(movie):
    searchResponse=searchInstance.movie(query=movie)
    id=searchResponse['results'][0]['id']
    movie=tmdb.Movies(id)
    overview=movie.info()['overview']
    return overview


In [50]:
for i in range(XTestTfidf.shape[0]):
    if i%60==0 and i!=0:
        print 'MOVIE: ',moviesWithOverviews[i]['title'],'PREDICTION: ',','.join(predictionsnb[i]), ';','Actual:',get_movie_genres(moviesWithOverviews[i]['title'])

MOVIE:  Justin and the Knights of Valour PREDICTION:  Thriller ; Actual: [{u'id': 16, u'name': u'Animation'}, {u'id': 12, u'name': u'Adventure'}]
MOVIE:  Igor PREDICTION:  Action,Thriller,Crime ; Actual: [{u'id': 16, u'name': u'Animation'}, {u'id': 35, u'name': u'Comedy'}, {u'id': 10751, u'name': u'Family'}, {u'id': 14, u'name': u'Fantasy'}]
MOVIE:  Terminus PREDICTION:  Adventure,Fantasy,Animation,Comedy,Thriller,Family ; Actual: [{u'id': 878, u'name': u'Science Fiction'}, {u'id': 53, u'name': u'Thriller'}, {u'id': 14, u'name': u'Fantasy'}]
MOVIE:  Donkey X PREDICTION:  Music ; Actual: [{u'id': 10751, u'name': u'Family'}, {u'id': 12, u'name': u'Adventure'}, {u'id': 16, u'name': u'Animation'}]
MOVIE:  Les Misérables PREDICTION:  Animation,Thriller,Crime,Family ; Actual: [{u'id': 18, u'name': u'Drama'}, {u'id': 10402, u'name': u'Music'}, {u'id': 10749, u'name': u'Romance'}]


So there is some overlap between predicted and actual genre values. Now we need to check the model using Evaluation Metrics .
### Here I am using Precision and recall metrics since this is a multi label problem, and conventionally precision and recall metrics are decent to be used when dealing with such problems

In [51]:
def precision_recall(gt,preds):
    TP=0
    FP=0
    FN=0
    for t in gt:
        if t in preds:
            TP+=1
        else:
            FN+=1
    for p in preds:
        if p not in gt:
            FP+=1
    if TP+FP==0:
        precision=0
    else:
        precision=TP/float(TP+FP)
    if TP+FN==0:
        recall=0
    else:
        recall=TP/float(TP+FN)
    return precision,recall

In [58]:
#Evaluation metric on Support Vector Machine
precs=[]
recs=[]
for i in range(len(testMovies)):
    if i%1==0:
        pos=testMovies[i]
        testMovie=moviesWithOverviews[pos]
        gtids=testMovie['genre_ids']
        gt=[]
        for g in gtids:
            gName=GenreIDToName[g]
            gt.append(gName)
#         print predictions[i],movies_with_overviews[i]['title'],gt
        a,b=precision_recall(gt,predictions[i])
        precs.append(a)
        recs.append(b)

print np.mean(np.asarray(precs)),np.mean(np.asarray(recs))

0.330065436771 0.531677704194


In [59]:
#Evaluation metric on Naive Bayes Classifier
precs=[]
recs=[]
for i in range(len(testMovies)):
    if i%1==0:
        pos=testMovies[i]
        testMovie=moviesWithOverviews[pos]
        gtids=testMovie['genre_ids']
        gt=[]
        for g in gtids:
            gName=GenreIDToName[g]
            gt.append(gName)
#         print predictions[i],movies_with_overviews[i]['title'],gt
        a,b=precision_recall(gt,predictionsnb[i])
        precs.append(a)
        recs.append(b)

print np.mean(np.asarray(precs)),np.mean(np.asarray(recs))

0.48893866021 0.521633554084


Since the mean precision and recall scores for our test samples are pretty good! Models seem to be acceptable.

Also, it can be infered that the Naive Bayes performs better than SVM in this scenario.
And as it's well known that Multinomial Bayes is used for dealing with the problem of "Document Classification", Naive bayes also shows us better results because the model here is designed similar to the problem of document classification as each overview of a movie resembles a document and based on content of each overview, we classify the movie in different genres.